# cuGraph Single Source Shortest Path

In this notebook, you will use GPU-accelerated graph analytics with cuGraph to identify the shortest path from node on the road network to every other node, both by distance, which we will demo, and by time, which you will implement. You will also visualize the results of your findings.

## Objectives

By the time you complete this notebook you will be able to:

- Use GPU-accelerated SSSP algorithm
- Use cuXfilter to create a heat map of average travel times

## Imports

In [1]:
import cudf
import cugraph as cg

import cuxfilter as cxf
from bokeh.palettes import Magma, Turbo256, Plasma256, Viridis256

## Loading Data

We start by loading the road graph data you prepared for constructing a graph with cuGraph, with the long unique `nodeid` replaced with simple (and memory-efficient) integers we call the `graph_id`.

In [2]:
road_graph = cudf.read_csv('./data/road_graph_2-09.csv', dtype=['int32', 'int32', 'float32'])
road_graph.head()

,src,dst,length
0,0,129165,44.0
1,1,1678323,70.0
2,1,2372610,18.0
3,1,2483057,40.0
4,2,2,55.0


Next we load the graph-ready data you prepared that uses amount of time traveled as edge weight.

In [3]:
speed_graph = cudf.read_csv('./data/road_graph_speed_2-09.csv', dtype=['int32', 'int32', 'float32'])
speed_graph.head()

,src,dst,length_s
0,0,129165,3.280848
1,1,1678323,5.219531
2,1,2372610,1.342165
3,1,2483057,2.982589
4,2,2,4.101060


Finally we import the full `road_nodes` data set, which we will use below for visualizations.

In [4]:
road_nodes = cudf.read_csv('./data/road_nodes_2-09.csv', dtype=['str', 'float32', 'float32', 'str'])
road_nodes = road_nodes.drop_duplicates() # again, some road nodes appeared on multiple map tiles in the original source
road_nodes.head()

,node_id,east,north,type
2589119,id000000F5-5180-4C03-B05D-B01352C54F89,432920.250,572547.375000,road end
1954117,id000003F8-9E09-4829-AD87-6DA4438D22D8,526616.375,189678.390625,junction
873541,id000010DA-C89A-4198-847A-6E62815E038A,336879.000,731824.000000,junction
1346912,id000017A0-1843-4BC7-BCF7-C943B6780839,380635.000,390153.000000,junction
1553458,id00001B2A-155F-4CD3-8E06-7677ADC6AF74,337481.000,350509.687500,junction


In [5]:
road_nodes.shape

(3078117, 4)

In [6]:
speed_graph.src.max()

3078116

## Construct Graph with cuGraph

Now that we have the well-prepped `road_graph` data, we pass it to cuGraph to create our graph data structure, which we can then use for accelerated analysis. In order to do so, we first use cuGraph to instantiate a `Graph` instance, and then pass the instance edge sources, edge destinations, and edge weights, currently the length of the roads.

In [7]:
G = cg.Graph()
%time G.from_cudf_edgelist(road_graph, source='src', destination='dst', edge_attr='length')

CPU times: user 91.9 ms, sys: 52 ms, total: 144 ms
Wall time: 144 ms


## Analyzing the Graph

First, we check the number of nodes and edges in our graph:

In [8]:
G.number_of_nodes()

3078117

In [9]:
G.number_of_edges()

3620793

We can also analyze the degrees of our graph nodes. We would expect, as before, that every node would have a degree of 2 or higher, since undirected edges count as two edges (one in, one out) for each of their nodes.

In [10]:
deg_df = G.degree()
deg_df['degree'].describe()[1:]

mean     4.689990
std      1.913452
min      2.000000
25%      2.000000
50%      6.000000
75%      6.000000
max     16.000000
Name: degree, dtype: float64

In [11]:
deg_df.head()

,degree,vertex
0,8,127763
1,8,127765
2,8,127768
3,8,127791
4,8,127816


We would also expect that every degree would be a multiple of 2, for the same reason. We check that there are no nodes with odd degrees (that is, degrees with a value of 1 modulo 2):

In [12]:
deg_df[deg_df['degree'].mod(2) == 1]

,degree,vertex


Observe for reference that some roads loop from a node back to itself:

In [13]:
road_graph.loc[road_graph.src == road_graph.dst]

,src,dst,length
4,2,2,55.0
145,62,62,108.0
293,124,124,67.0
471,196,196,26.0
571,240,240,44.0
...,...,...,...
7216602,3077469,3077469,78.0
7216735,3077519,3077519,111.0
7216849,3077567,3077567,69.0
7217091,3077670,3077670,30.0


## Single Source Shortest Path

To demo the Single Source Shortest Path (SSSP) algorithm, we will start with the node with the highest degree. First we obtain its `graph_id`, reported by the `degree` method as `vertex`:

In [14]:
demo_node = deg_df.nlargest(1, 'degree')
demo_node_graph_id = demo_node['vertex'].iloc[0]
demo_node_graph_id

652907

We can now call `cg.sssp`, passing it the entire graph `G`, and the `graph_id` for our selected vertex. Doing so will calculate the shortest path, using the road length weights we have provided, to *every* other node in the graph - millions of paths, in seconds:

In [15]:
%time shortest_distances_from_demo_node = cg.sssp(G, demo_node_graph_id)
shortest_distances_from_demo_node.head()

CPU times: user 12.4 s, sys: 71.8 ms, total: 12.5 s
Wall time: 12.4 s


,distance,vertex,predecessor
0,215398.0,452873,200589
1,147424.0,452874,633387
2,167215.0,452876,1641914
3,211350.0,452893,820362
4,151358.0,453033,2635012


In [ ]:
# Limiting to those nodes that were connected (within ~4.3 billion meters because
# cg.sssp uses the max int value for unreachable nodes, such as those on different islands)
shortest_distances_from_demo_node['distance'].loc[shortest_distances_from_demo_node['distance'] < 2**32].describe()[1:]

## Exercise: Analyze a Graph with Time Weights

For this exercise, you are going to analyze the graph of GB's roads, but this time, instead of using raw distance for a road's weights, you will be using how long it will take to travel along the road.

### Step 1: Construct the Graph

Construct a cuGraph graph called `G_ex` using the sources and destinations found in `speed_graph`, along with length in seconds values for the edges' weights.

In [16]:
speed_graph.head()

,src,dst,length_s
0,0,129165,3.280848
1,1,1678323,5.219531
2,1,2372610,1.342165
3,1,2483057,2.982589
4,2,2,4.101060


In [17]:
G_ex = cg.Graph()

%time G_ex.from_cudf_edgelist(speed_graph, source='src', destination='dst', edge_attr='length_s')

CPU times: user 87.4 ms, sys: 38.3 ms, total: 126 ms
Wall time: 125 ms


#### Solution

In [ ]:
# %load solutions/construct_graph
G_ex = cg.Graph()
G_ex.from_cudf_edgelist(speed_graph, source='src', destination='dst', edge_attr='length_s')


### Step 2: Get Travel Times From a Node to All Others

Choose one of the nodes and calculate the time it would take to travel from it to all other nodes via SSSP, calling the results `ex_dist`.

In [19]:
nodeexx = degex.nlargest(2, 'degree')
nodeex = nodeexx[1:]
nodeex['vertex'].iloc[0]

1252792

In [20]:
degex = G_ex.degree()
nodeexx = degex.nlargest(2, 'degree')
nodeex = nodeexx[1:]
ex_dist = cg.sssp(G_ex, nodeex)
ex_dist

,distance,vertex,predecessor
0,10479.518555,12310,1659158
1,11747.610352,12375,750622
2,1794.442993,12379,1964986
3,551.033264,12383,2150224
4,10329.327148,12405,272679
...,...,...,...
3078112,10113.742188,3066279,74311
3078113,7544.033203,3066295,157131
3078114,6876.803711,3066246,1262774
3078115,6427.470703,3066282,2396026


#### Solution

In [21]:
ex_dist['distance'].loc[ex_dist['distance'] < 2**32].describe()[1:]

mean     9451.564306
std      6721.476632
min         0.000000
25%      4511.428467
50%      8444.589844
75%     11944.945801
max     37873.683594
Name: distance, dtype: float64

In [22]:
ex_node['vertex'].iloc[0]

NameError: name 'ex_node' is not defined

In [ ]:
# %load solutions/travel_times
# If you have time, see what the SSSP visualization looks like starting from nodes at different extreme coordinates,
# or one of the end nodes of an especially long edge, or even one of the nodes unreachable from the main road network.
ex_deg = G_ex.degree()
ex_node = ex_deg.nlargest(1, 'degree')

%time ex_dist = cg.sssp(G_ex, ex_node['vertex'].iloc[0])

# limiting to those nodes that were connected (within ~4.3 billion seconds; .sssp uses the max int value for unconnected nodes)
ex_dist['distance'].loc[ex_dist['distance'] < 2**32].describe()[1:]


### Step 3: Visualize the Node Travel Times

In order to create a graphic showing the road network by travel time from the selected node, we first need to align the just-calculated distances with their original nodes. For that, we use the mapping from `node_id` strings to their `graph_id` integers.

In [23]:
mapping = cudf.read_csv('./data/node_graph_map.csv')
mapping.head()

,node_id,graph_id
0,id000000F5-5180-4C03-B05D-B01352C54F89,0
1,id000003F8-9E09-4829-AD87-6DA4438D22D8,1
2,id000010DA-C89A-4198-847A-6E62815E038A,2
3,id000017A0-1843-4BC7-BCF7-C943B6780839,3
4,id00001B2A-155F-4CD3-8E06-7677ADC6AF74,4


We see that the `sssp` algorithm has put the `graph_id`s in the `vertex` column, so we will merge on that.

In [24]:
ex_dist.head()

,distance,vertex,predecessor
0,10479.518555,12310,1659158
1,11747.610352,12375,750622
2,1794.442993,12379,1964986
3,551.033264,12383,2150224
4,10329.327148,12405,272679


In [25]:
road_nodes = road_nodes.merge(mapping, on='node_id')
road_nodes = road_nodes.merge(ex_dist, left_on='graph_id', right_on='vertex')
road_nodes.head()

,node_id,east,north,type,graph_id,distance,vertex,predecessor
0,id014E7D40-3293-4A97-BFFD-A3E12DE8E1E6,256525.109375,670072.500000,junction,15760,21281.820312,15760,1885289
1,id014E8247-309E-431D-9A75-B5A76B8AD438,511185.437500,333940.250000,road end,15761,6380.525879,15761,2661311
2,id014E86A9-2215-42D3-8F5F-BE798CE5C614,169486.062500,33502.121094,road end,15762,16214.071289,15762,2887600
3,id014E88EE-AED2-4782-9FB1-2080272B15E5,395897.187500,299463.531250,junction,15763,6940.216309,15763,3028181
4,id014E9024-51F5-40F6-A871-B10EE50D972D,483820.375000,155520.734375,road end,15764,2263.418701,15764,2856596


Next, we select those columns we are going to use for the visualization.

For color-scaling purposes, we get rid of the unreachable nodes with their extreme distances, and we invert the distance numbers so that brighter pixels indicate closer locations.

In [26]:
gdf = road_nodes[['east', 'north', 'distance']]
gdf = gdf[gdf['distance'] < 2**32]
gdf['distance'] = gdf['distance'].pow(1/2).mul(-1)

Otherwise, this visualization will be largely similar to the scatter plots we made to visualize the population, but instead of coloring by point density as in those cases, we will color by mean travel time to the nodes within a pixel.

In [27]:
cxf_data = cxf.DataFrame.from_dataframe(gdf)

In [28]:
chart_width = 600
heatmap_chart = cxf.charts.datashader.scatter(x='east', y='north', 
                                              width=chart_width, 
                                              height=int((gdf['north'].max() - gdf['north'].min()) / 
                                                         (gdf['east'].max() - gdf['east'].min()) *
                                                          chart_width),
                                              #palettes=Plasma256, # try also Turbo256, Viridis256, Magma
                                              #pixel_shade_type='linear', # can also be log, cbrt
                                              aggregate_col='distance',
                                              aggregate_fn='mean',
                                              #point_shape='square',
                                              point_size=1)

In [29]:
dash = cxf_data.dashboard([heatmap_chart], theme=cxf.themes.dark, data_size_widget=True)

heatmap_chart.view()

Card(sizing_mode='scale_width', title='Scatter plot f..., width=600)
    [0] HoloViews(DynamicMap, height=400, sizing_mode='stretch_both')

In [78]:
%%js
var host = window.location.host;
element.innerText = "'http://"+host+"'";

<IPython.core.display.Javascript object>

Set `my_url` in the next cell to the value just printed, making sure to include the quotes:

In [79]:
my_url ="http://dli-604a4aa51b37-3dea5f.aws.labs.courses.nvidia.com" # TODO: Set this value to the print out of the cell above, including the quotes.
dash.show(my_url + "/lab", port=8789)

Dashboard running at port 8789


Row(sizing_mode='stretch_both')
    [0] Button(button_type='success', name='open cuxfilter d..., sizing_mode='stretch_both')

... and you can run the next cell to generate a link to the dashboard:

In [80]:
%%js
var host = window.location.host;
var url = 'http://'+host+'/lab/proxy/8789/';
element.innerHTML = '<a style="color:blue;" target="_blank" href='+url+'>Open Dashboard</a>';

<IPython.core.display.Javascript object>

In [81]:
dash.stop()

## Next

This concludes the second section of the workshop. In the third section, you'll put the skills you've learned in this workshop to the test by helping over several simulated days to address a national epidemic.

<br>
<div align="center"><h2>Optional: Restart the Kernel</h2></div>

If you plan to do additional work in other notebooks, please restart the kernel:

In [30]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}